### Code GAN

In [ ]:
!pip install torchmetrics torchquad statsmodels
!apt install texlive
!sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended cm-super

In [ ]:
import torch.nn as nn
import torch
from torchmetrics import Accuracy
# from torchvision.datasets import FashionMNIST
# from torchvision import transforms
# from torch.utils.data import  DataLoader
from torch.functional import F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['text.usetex'] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams['font.size'] = 15
import sys
from sklearn.utils import shuffle

cuda = True if torch.cuda.is_available() else False
from torch.autograd import Variable
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
torch.manual_seed(0)

#### Create Data

In [ ]:
class DataModule:
    def __init__(self,dim, normalize=True, **kwargs):
        # a 3-D X : (batchs, numbers, 1)
        # and we will follow the -π to π for initial experiments
        if kwargs['f'] == 'delta':
            self.p = self.delta()
        elif kwargs['f'] == 'gauss':
            self.X = np.random.normal(loc=kwargs['mu'], scale=kwargs['sigma'], size=dim)
            if normalize:
                self.scale = (self.X.max() - self.X.min())
                self.X = self.X / self.scale
            self.p = self.gauss(kwargs['mu'], kwargs['sigma'])
        elif kwargs['f'] == 'uniform':
            self.X = np.random.uniform(kwargs['min'], kwargs['max'], size=dim)
            if normalize:
                self.scale = (self.X.max() - self.X.min())
                self.X = self.X / self.scale
            self.p = self.uniform(kwargs['min'], kwargs['max'])
        train_ratio = int(0.8*len(self.X))
        self.X_train, self.X_test = self.X[:train_ratio], self.X[train_ratio:]
        self.X_train, self.X_test  = Variable(Tensor(self.X_train)),Variable(Tensor(self.X_test))

    def delta(self):
        dim = self.X.shape
        return np.where(self.X == 0, np.inf*np.ones(dim), np.zeros(dim))

    def gauss(self, mu, sigma):
        return np.exp(-0.5*((self.X-mu)/sigma)**2) / (np.sqrt(2*np.pi)*sigma)

    def uniform(self, min, max):
        dim = self.X.shape
        return 1 / (max - min) * np.ones(dim)

#### Generator and Discriminator

In [ ]:
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
                nn.Linear(1, 4),
                nn.LeakyReLU(0.01),
                nn.Linear(4,4),
                nn.LeakyReLU(0.01),
                nn.Linear(4,4),
                nn.LeakyReLU(0.01),
                nn.Linear(4, 1),
                nn.Tanh()
            )

    def forward(self, z):
        y_gen = self.model(z)
        return y_gen


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1, 8),   #8,
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.Sigmoid(),
        )

    def forward(self, fn):
        validity = self.model(fn)
        return validity


adversarial_loss = nn.BCELoss()


## **$\mathcal{N}(0,1) \to \mathcal{N}(-200,1)$**

### Vanilla GAN

In [ ]:
!rm -r Log
!mkdir Log
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)



data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 1000
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.normal(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        real_loss = adversarial_loss(discriminator(f), lbl_one)
        fake_loss = adversarial_loss(discriminator(f_tilde), lbl_zero)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = adversarial_loss(discriminator(f_tilde.detach()), lbl_one)
        g_loss.backward()
        optimizer_G.step()
        if (e%10 == 0):
            torch.save(generator, "./Log/Generaor_%d"%(e))

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

    
plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis

In [ ]:
noisetype = "Normal(0,1)"
GAN = "Vanilla GAN"

z = Variable(Tensor(np.random.normal(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 1) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
res = stats.anderson(x1, dist='norm')
print(res)

In [ ]:
with open("./M/RUN2/N/VGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### W-GAN

In [ ]:

generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 200
gloss, dloss = [], []
clip_value = 0.2
n_critic = 7


batches_done = 0
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        z = Variable(Tensor(np.random.normal(size=dimn)))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()


        # Generate a batch of images
        f_tilde = generator(z).detach()
        # Adversarial loss
        d_loss = -torch.mean(discriminator(f)) + torch.mean(discriminator(f_tilde))

        d_loss.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

        # Train the generator every n_critic iterations
        if e % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            f_tilde = generator(z)
            # Adversarial loss
            g_loss = -torch.mean(discriminator(f_tilde))

            g_loss.backward()
            optimizer_G.step()

        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))

        batches_done += 1
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of W-GAN

In [ ]:
noisetype = "Normal(0,1)"
GAN = "W-GAN"

z = Variable(Tensor(np.random.normal(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 1) \n after %d Epochs using %s GAN'%(noisetype, 1000, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/N/WGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### LS-GAN


In [ ]:
'https://github.com/meliketoy/LSGAN.pytorch/blob/master/main.py'
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)



data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1, normalize=False)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.normal(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        errD_real = 0.5 * torch.mean((discriminator(f) - lbl_one)**2)
        errD_fake = 0.5 * torch.mean((discriminator(f_tilde) - lbl_zero)**2)
        d_loss = (errD_fake + errD_real) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = 0.5 * torch.mean((discriminator(f_tilde.detach()) - lbl_one)**2) # criterion(output, label)
        g_loss.backward()
        optimizer_G.step()

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of LSGAN

In [ ]:
noisetype = "Normal(0,1)"
GAN = "LSGAN"

z = Variable(Tensor(np.random.normal(size=(1000,1))))
f_tilde = (generator(z)).flatten()
x1 = f_tilde.detach().cpu().numpy()
x2 = Xt_test.flatten().detach().cpu().numpy()
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 1) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/N/LSGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()


## End of Part I
---



---



---



## **$\mathcal{U}(0,1) \to \mathcal{N}(-200,1)$**

### Vanilla GAN


In [ ]:
!rm -r Log2
!mkdir Log2
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)



data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 1000
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.random(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        real_loss = adversarial_loss(discriminator(f), lbl_one)
        fake_loss = adversarial_loss(discriminator(f_tilde), lbl_zero)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = adversarial_loss(discriminator(f_tilde.detach()), lbl_one)
        g_loss.backward()
        optimizer_G.step()
        if (e%10 == 0):
            torch.save(generator, "./Log/Generaor_%d"%(e))

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis

In [ ]:
noisetype = "Uniform(0,1)"
GAN = "Vanilla GAN"

z = Variable(Tensor(np.random.random(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 1) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/U(0,1)toN(-200,1)/VGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### W-GAN

In [ ]:

generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
clip_value = 0.2
n_critic = 7


batches_done = 0
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        z = Variable(Tensor(np.random.random(size=dimn)))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()


        # Generate a batch of images
        f_tilde = generator(z).detach()
        # Adversarial loss
        d_loss = -torch.mean(discriminator(f)) + torch.mean(discriminator(f_tilde))

        d_loss.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

        # Train the generator every n_critic iterations
        if e % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            f_tilde = generator(z)
            # Adversarial loss
            g_loss = -torch.mean(discriminator(f_tilde))

            g_loss.backward()
            optimizer_G.step()

        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))

        batches_done += 1
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of W-GAN

In [ ]:
noisetype = "Uniform(0,1)"
GAN = "W-GAN"

z = Variable(Tensor(np.random.random(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 1) \n after %d Epochs using %s GAN'%(noisetype, 1400, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/U(0,1)toN(-200,1)/WGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### LS-GAN

In [ ]:
'https://github.com/meliketoy/LSGAN.pytorch/blob/master/main.py'
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


# data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1, normalize=True)
data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1, normalize=False)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.random(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        errD_real = 0.5 * torch.mean((discriminator(f) - lbl_one)**2)
        errD_fake = 0.5 * torch.mean((discriminator(f_tilde) - lbl_zero)**2)
        d_loss = (errD_fake + errD_real) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = 0.5 * torch.mean((discriminator(f_tilde.detach()) - lbl_one)**2) # criterion(output, label)
        g_loss.backward()
        optimizer_G.step()

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of LSGAN

In [ ]:
noisetype = "Uniform(0,1)"
GAN = "LSGAN"

z = Variable(Tensor(np.random.random(size=(1000,1))))
f_tilde = (generator(z)).flatten() # * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() # * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 1) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/U(0,1)toN(-200,1)/LSGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

## End of Part II

---

---

## **$\mathcal{N}(0,1) \to \mathcal{N}(-200,5)$**

### Vanilla GAN


In [ ]:
!rm -r Log2
!mkdir Log2
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)



data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=5)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 1000
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.normal(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        real_loss = adversarial_loss(discriminator(f), lbl_one)
        fake_loss = adversarial_loss(discriminator(f_tilde), lbl_zero)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = adversarial_loss(discriminator(f_tilde.detach()), lbl_one)
        g_loss.backward()
        optimizer_G.step()
        if (e%10 == 0):
            torch.save(generator, "./Log/Generaor_%d"%(e))

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis

In [ ]:
noisetype = "Normal(0,1)"
GAN = "Vanilla GAN"

z = Variable(Tensor(np.random.normal(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 5) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/N-N(m,s)/VGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### W-GAN

In [ ]:

generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=5)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
clip_value = 0.2
n_critic = 7


batches_done = 0
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        z = Variable(Tensor(np.random.normal(size=dimn)))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()


        # Generate a batch of images
        f_tilde = generator(z).detach()
        # Adversarial loss
        d_loss = -torch.mean(discriminator(f)) + torch.mean(discriminator(f_tilde))

        d_loss.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

        # Train the generator every n_critic iterations
        if e % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            f_tilde = generator(z)
            # Adversarial loss
            g_loss = -torch.mean(discriminator(f_tilde))

            g_loss.backward()
            optimizer_G.step()

        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))

        batches_done += 1
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of W-GAN

In [ ]:
noisetype = "Normal(0,1)"
GAN = "W-GAN"

z = Variable(Tensor(np.random.normal(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 5) \n after %d Epochs using %s GAN'%(noisetype, 1400, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/N-N(m,s)/WGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### LS-GAN

In [ ]:
'https://github.com/meliketoy/LSGAN.pytorch/blob/master/main.py'
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


# data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1, normalize=True)
data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=5, normalize=False)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.normal(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        errD_real = 0.5 * torch.mean((discriminator(f) - lbl_one)**2)
        errD_fake = 0.5 * torch.mean((discriminator(f_tilde) - lbl_zero)**2)
        d_loss = (errD_fake + errD_real) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = 0.5 * torch.mean((discriminator(f_tilde.detach()) - lbl_one)**2) # criterion(output, label)
        g_loss.backward()
        optimizer_G.step()

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of LSGAN

In [ ]:
noisetype = "Normal(0,1)"
GAN = "LSGAN"

z = Variable(Tensor(np.random.normal(size=(1000,1))))
f_tilde = (generator(z)).flatten() # * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() # * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Target Data')
# plt.ylabel('Generated Data')
# plt.title('Given %s Noise Reach Normal(-200, 5) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/N-N(m,s)/LSGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

## End of Part III

---

---

## **$\mathcal{U}(0,1) \to \mathcal{N}(0,1)$**

### Vanilla GAN


In [ ]:
!rm -r Log2
!mkdir Log2
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)



data_loader = DataModule(dim=(100,50,1), f='gauss', mu=0, sigma=1)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 1000
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.random(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        real_loss = adversarial_loss(discriminator(f), lbl_one)
        fake_loss = adversarial_loss(discriminator(f_tilde), lbl_zero)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = adversarial_loss(discriminator(f_tilde.detach()), lbl_one)
        g_loss.backward()
        optimizer_G.step()
        if (e%10 == 0):
            torch.save(generator, "./Log/Generaor_%d"%(e))

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]\r"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis

In [ ]:
noisetype = "Uniform(0,1)"
GAN = "Vanilla GAN"
import statsmodels.api as sm 
import pylab as py 

z = Variable(Tensor(np.random.random(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
# plt.xlabel('Generated Data')
# plt.ylabel('Target Data')
# plt.title('Given %s Noise Reach Normal(0, 1) \n after %d Epochs using %s'%(noisetype, epochs, GAN))
plt.show()

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/U(0,1)toN(0,1)/VGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### W-GAN

In [ ]:

generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


data_loader = DataModule(dim=(100,50,1), f='gauss', mu=0, sigma=1)
# data = DataModule(dim=(10,50,1), f='uniform', min=-12, max=500)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
clip_value = 0.2
n_critic = 7


batches_done = 0
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        z = Variable(Tensor(np.random.random(size=dimn)))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()


        # Generate a batch of images
        f_tilde = generator(z).detach()
        # Adversarial loss
        d_loss = -torch.mean(discriminator(f)) + torch.mean(discriminator(f_tilde))

        d_loss.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

        # Train the generator every n_critic iterations
        if e % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            f_tilde = generator(z)
            # Adversarial loss
            g_loss = -torch.mean(discriminator(f_tilde))

            g_loss.backward()
            optimizer_G.step()

        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))

        batches_done += 1
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of W-GAN

In [ ]:
noisetype = "Uniform(0,1)"
GAN = "W-GAN"

z = Variable(Tensor(np.random.random(size=(1000,1))))
f_tilde = (generator(z)).flatten() * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/U(0,1)toN(0,1)/WGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

### LS-GAN

In [ ]:
'https://github.com/meliketoy/LSGAN.pytorch/blob/master/main.py'
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3)
optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=1e-3)


# data_loader = DataModule(dim=(100,50,1), f='gauss', mu=-200, sigma=1, normalize=True)
data_loader = DataModule(dim=(100,50,1), f='gauss', mu=0, sigma=1, normalize=False)
Xt_train, Xt_test = data_loader.X_train, data_loader.X_test

In [ ]:

epochs = 400
gloss, dloss = [], []
for e in range(epochs):
    for b, f in enumerate(data_loader.X_train):
        dimn = (f.size(dim=0),  f.size(dim=1))

        lbl_one = Variable(Tensor(np.ones(dimn)), requires_grad=False)
        z = Variable(Tensor(np.random.random(size=dimn)))
        f_tilde = generator(z)

        optimizer_D.zero_grad()
        lbl_zero = Variable(Tensor(np.zeros(dimn)), requires_grad=False)
        errD_real = 0.5 * torch.mean((discriminator(f) - lbl_one)**2)
        errD_fake = 0.5 * torch.mean((discriminator(f_tilde) - lbl_zero)**2)
        d_loss = (errD_fake + errD_real) / 2
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        g_loss = 0.5 * torch.mean((discriminator(f_tilde.detach()) - lbl_one)**2) # criterion(output, label)
        g_loss.backward()
        optimizer_G.step()

        # accuracy = Accuracy()
        # b_acc = accuracy(discriminator(f_tilde.detach())>=0.5, lbl_zero.type(torch.int8))
        sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%
                         (e+1, epochs, b+1, len(Xt_train), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
        # sys.stdout.write("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G Loss: %f] [Generated: %f]"%\
                        # (e+1, epochs, b+1, len(data_loader), d_loss.item(), g_loss.item(), torch.mean(f_tilde)))
    gloss.append(g_loss.item())
    dloss.append(d_loss.item())

plt.plot(range(epochs), gloss)
plt.plot(range(epochs), dloss)
plt.show()

#### Analysis of LSGAN

In [ ]:
noisetype = "Uniform(0,1)"
GAN = "LSGAN"

z = Variable(Tensor(np.random.random(size=(1000,1))))
f_tilde = (generator(z)).flatten() # * data_loader.scale
x1 = f_tilde.cpu().detach().numpy()
x2 = Xt_test.flatten().cpu().detach().numpy() # * data_loader.scale
import scipy.stats as stats
fig = plt.figure(figsize=(5,3))
x1.sort()
x2.sort()
plt.scatter(x1, x2, label='Direct QQ-plot on Tar vs Gen', color='blue')
plt.show()
'https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py'

In [ ]:
# x1 = (x1+200)/500
hist = torch.histc(torch.tensor(x1), bins=10, min=x1.min(), max=x1.max())
bins = 10
x = range(bins)
plt.bar(x, hist, align='center')
plt.xlabel('Bins')

In [ ]:
stats.anderson(x1, dist='norm')

In [ ]:
with open("./M/RUN2/U(0,1)toN(0,1)/LSGAN.txt", "w") as f:
    for x, y in zip(x1, x2):
        f.write(str(x) + ',' + str(y)+ '\n')
f.close()

## End of Part IV

---

---